In [1]:
import numpy as np
import os
import pandas as pd
from rich import inspect

from utils import *

console = setup_rich()

In [2]:
filenames = ["discord-4nkq7191ray", "discord-fv8dqxpzsem"]
data_dir = ".data/csv/"
filenames = [os.path.join(data_dir, f"{f}.csv") for f in filenames]
train_data = pd.read_csv(filenames[1])
test_data = pd.read_csv(filenames[0])
train_data.head()

,text
0,Hehe
1,"20220127_210848.jpg"",""prependIcon"":""attachment..."
2,I think I'm gonna stream playing im excited
3,Time to impulse buy
4,Wait is pokemon diamond out?


# Finetuning T.E.D.D.Y.

In [3]:
# Load model directly
import warnings

import torch
from transformers import *

warnings.filterwarnings("ignore")


model_name = "microsoft/Phi-3-mini-4k-instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device="cpu",
)
pipe

c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
loading file tokenizer.model from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\tokenizer.model
loading file added_tokens.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapsh

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at C:\Users\nicol\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000002DDA4F2C750>

In [4]:
# pipe("Can you explain LangChain?")

In [5]:
inspect(pipe)

╭───────── <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000002DDA4F2C750> ──────────╮
│ def (text_inputs, **kwargs):                                                                                    │
│                                                                                                                 │
│ Language generation pipeline using any `ModelWithLMHead`. This pipeline predicts the words that will follow a   │
│ specified text prompt. When the underlying model is a conversational model, it can also accept one or more      │
│ chats,                                                                                                          │
│ in which case the pipeline will operate in chat mode and will continue the chat(s) by adding its response(s).   │
│ Each chat takes the form of a list of dicts, where each dict contains "role" and "content" keys.                │
│                                                                                                                 │
│       binary_output = False                                                                                     │
│          call_count = 0                                                                                         │
│ default_input_names = None                                                                                      │
│              device = device(type='cpu')                                                                        │
│   feature_extractor = None                                                                                      │
│           framework = 'pt'                                                                                      │
│     image_processor = None                                                                                      │
│           modelcard = None                                                                                      │
│                task = 'text-generation'                                                                         │
│         torch_dtype = torch.float32                                                                             │
│           XL_PREFIX = "\n    In 1991, the remains of Russian Tsar Nicholas II and his family (except for Alexei │
│                       and Maria) are discovered. The\n    voice of Nicholas's young son, Tsarevich Alexei       │
│                       Nikolaevich, narrates the remainder of the story. 1883 Western\n    Siberia, a young      │
│                       Grigori Rasputin is asked by his father and a group of men to perform magic. Rasputin has │
│                       a vision\n    and denounces one of the men as a horse thief. Although his father          │
│                       initially slaps him for making such an\n    accusation, Rasputin watches as the man is    │
│                       chased outside and beaten. Twenty years later, Rasputin sees a vision of\n    the Virgin  │
│                       Mary, prompting him to become a priest. Rasputin quickly becomes famous, with people,     │
│                       even a bishop,\n    begging for his blessing. <eod> </s> <eos>\n    "                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
sample = train_data.sample(1)["text"].values[0]
print(sample)


one sec


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_dataset

Dataset({
    features: ['text'],
    num_rows: 51305
})

In [8]:
sample = train_data.sample(1)["text"].values[0]
console.print(sample)
input_ids = tokenizer.encode(sample, return_tensors="pt")
input_ids

one day you'll actually join

tensor([[  697,  2462,   366, 29915,   645,  2869,  5988]])

In [9]:
texts = train_data["text"].tolist()

In [10]:
texts_encodings = tokenizer(texts[:5], truncation=True, padding=True)

In [11]:
texts_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
from peft import get_peft_model, LoraConfig, TaskType


In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/51305 [00:00<?, ? examples/s]

Map:   0%|          | 0/4245 [00:00<?, ? examples/s]

In [14]:
inspect(train_dataset)

╭─────────────────────────────────── <class 'datasets.arrow_dataset.Dataset'> ────────────────────────────────────╮
│ A Dataset backed by an Arrow table.                                                                             │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Dataset({                                                                                                   │ │
│ │ │   features: ['text', 'input_ids', 'attention_mask'],                                                      │ │
│ │ │   num_rows: 51305                                                                                         │ │
│ │ })                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│       builder_name = None                                                                                       │
│        cache_files = []                                                                                         │
│           citation = ''                                                                                         │
│       column_names = ['text', 'input_ids', 'attention_mask']                                                    │
│        config_name = None                                                                                       │
│               data = InMemoryTable                                                                              │
│                      text: string                                                                               │
│                      input_ids: list<item: int32>                                                               │
│                        child 0, item: int32                                                                     │
│                      attention_mask: list<item: int8>                                                           │
│                        child 0, item: int8                                                                      │
│                      ----                                                                                       │
│                      text:                                                                                      │
│                      [["Hehe","20220127_210848.jpg","prependIcon":"attachment"},{"text":"","mediaUrl":"/server… │
│                      think I'm gonna stream playing im excited","Time to impulse buy","Wait is pokemon diamond  │
│                      out?",...,"I also do like this has turned into a purity testing @Buys Bitcoin with         │
│                      EBT#1863 's right wingness, I didn't mean to","It's not hard for many ppl in here","Nope.  │
│                      Except for abstinence tho","Ah kk gotcha","Wait so would it or would it not be immoral for │
│                      the girl to then go get an abortion?"],["So what about in cases where birth control        │
│                      preventatives break? I assume the conclusion would be the same right?","I have wondered    │
│                      this tho: why include rape? I understand defects but if the idea is about saving lives why │
│                      would the rape matter? @Buys Bitcoin with EBT#1863","me being fascist?","@Buys Bitcoin     │
│                      with EBT#1863","LOL",...,"YOU THOUGHT I WAS JOKING","LOL","If I ever legally and feasible  │
│                      get my hands on human flesh I will try it and report back","Hmmm","Nope"],...,["black on   │
│                      black hate is REAL","DUDE i dated

# Training

In [15]:
output_dir = ".data/outputs/"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    remove_unused_columns=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

PyTorch: setting up devices
***** Running training *****
  Num examples = 51,305
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 19,242
  Number of trainable parameters = 3,821,079,552


  0%|          | 0/19242 [00:00<?, ?it/s]

You are not running the flash-attention implementation, expect numerical differences.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\nicol\AppData\Local\Temp\ipykernel_19328\3063029597.py:20 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\nicol\\AppData\\Local\\Temp\\ipykernel_19328\\3063029597.py'                         │
│                                                                                                  │
│ c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\trainer.py:1932 in train    │
│                                                                                                  │
│   1929 │   │   │   finally:                                                                      │
│   1930 │   │   │   │   hf_hub_utils.enable_progress_bars()                                       │
│   1931 │   │   else:                                                                             │
│ ❱ 1932 │   │   │   return inner_training_loop(                                                   │
│   1933 │   │   │   │   args=args,                                                                │
│   1934 │   │   │   │   resume_from_checkpoint=resume_from_checkpoint,                            │
│   1935 │   │   │   │   trial=trial,                                                              │
│                                                                                                  │
│ c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\trainer.py:2268 in          │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   2265 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   2266 │   │   │   │                                                                             │
│   2267 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 2268 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   2269 │   │   │   │                                                                             │
│   2270 │   │   │   │   if (                                                                      │
│   2271 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\trainer.py:3307 in          │
│ training_step                                                                                    │
│                                                                                                  │
│   3304 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   3305 │   │                                                                                     │
│   3306 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 3307 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   3308 │   │                                                                                     │
│   3309 │   │   del inputs                                                                        │
│   3310                                                                                           │
│                                                                                                  │
│ c:\Users\nicol\Anaconda3\envs\llm-dev\Lib\site-packages\transformers\trainer.py:3338 in          │
│ compute_loss                                               

In [ ]:
import math

eval_results = trainer.evaluate()
console.print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")